In [1]:
from time import sleep

import numpy as np
import torch
from torch import nn
import gym
from gym.envs.registration import register
from stable_baselines3 import *
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import wandb
from wandb.integration.sb3 import WandbCallback

from gsnake.env import GoogleSnakeEnv
from gsnake.configs import GoogleSnakeConfig

register(
    id='GoogleSnake-v1',
    entry_point=GoogleSnakeEnv,
    max_episode_steps=500,
)

pygame 2.1.0 (SDL 2.0.16, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
name = 'DQN_MIP_time_nch_obsfix_50M'

config = GoogleSnakeConfig(
    multi_channel=True,
    # reward_mode='basic',
    reward_mode='time_constrained',
    reward_scale=1,
    n_foods=3
)
run = wandb.init(
    job_type='train', config=config.__dict__,
    project='RL2',
    tags=[name.split('_')[0], 'gsnake'],
    name=name,
    sync_tensorboard=True,
    monitor_gym='False'
)
# Parallel environments
env = make_vec_env("GoogleSnake-v1", n_envs=10, env_kwargs={'config':config})
model = DQN("MultiInputPolicy", env, verbose=0, tensorboard_log=f'runs/{run.id}')

model.learn(total_timesteps=50_000_000, callback=WandbCallback(verbose=2), progress_bar=True)
run.finish()

model.save(f'{name}.pt')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dev-jahn. Use `wandb login --relogin` to force relogin


Output()

In [6]:
####################################################################
# Human evaluation
####################################################################
model = PPO.load("PPO_MLP_time.pt")
config = GoogleSnakeConfig(
    # reward_mode='basic',
    multi_channel=True,
    reward_mode='time_constrained',
    reward_scale=1,
    n_foods=3
)
env = GoogleSnakeEnv(config, 42, 'gui')
obs = env.reset()
try:
    while True:
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        env.render()
        sleep(0.5)
except KeyboardInterrupt:
    print('Terminated')
finally:
    env.close()

Terminated
